In [1]:
import numpy as np
import pandas as pd

import pymysql
import dotenv
import os
import pickle
import seaborn as sns
import matplotlib

from datetime import timedelta

import matplotlib.pyplot as plt

# Normalization/Standardization
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout, Conv1D, GRU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.saving import save_model

# 경고 무시 코드 추가
import warnings
warnings.filterwarnings('ignore')

def pltconfig_default() :
  sns.reset_defaults()
  %matplotlib inline

pltconfig_default()

matplotlib.rcParams

matplotlib.rcParams['font.family']

current_font_list = matplotlib.rcParams['font.family']

font_path = 'C:\\Windows\\Fonts\\batang.ttc'

kfont = matplotlib.font_manager.FontProperties(fname=font_path).get_name()

matplotlib.rcParams['font.family'] = [kfont] + current_font_list

In [2]:
St_NotEncode_data = pd.read_pickle("StandardScalar_final_data")

print(St_NotEncode_data.shape)

(17422, 9)


In [3]:
# Feature와 Label 분리하기
def Feature_Label(datafile) :
    X = datafile.iloc[:,:-1]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SNE_X, SNE_y = Feature_Label(St_NotEncode_data)
print(SNE_X.shape, SNE_y.shape)
SNE_X_train, SNE_X_test, SNE_y_train, SNE_y_test = train_test_split(SNE_X, SNE_y, test_size=0.2, random_state=10, shuffle=False)
print(SNE_X_train.shape, SNE_X_test.shape, SNE_y_train.shape, SNE_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 8) (17422,)
(13937, 8) (3485, 8) (13937,) (3485,)


In [4]:
# X 값 Window Dataset 구성
ds_x = tf.data.Dataset.from_tensor_slices(SNE_X_train).window(WINDOW_SIZE, stride=1, shift=1, drop_remainder=True)  # 여기서는 X값만 별도로 구성하므로 WINDOW_SIZE 값에 1을 더하지 않는다!
ds_x = ds_x.flat_map(lambda x : x.batch(WINDOW_SIZE))

In [5]:
# y 값 Window Dataset 구성
ds_y = tf.data.Dataset.from_tensor_slices(SNE_y_train[WINDOW_SIZE:])

In [6]:
train_data = tf.data.Dataset.zip((ds_x, ds_y)).batch(BATCH_SIZE)

In [7]:
for x, y in train_data.take(1) :
  print(x[:3])
  print()
  print(y[:3])

tf.Tensor(
[[[-0.11352868 -0.09386187 -0.6825146   0.09445245 -0.66581843
   -1.23511514 -0.21825161 -1.53935952]
  [-0.11352868 -0.9225242  -1.06038321  0.4051263  -0.66581843
   -1.23511514 -0.21825161 -1.57681737]
  [-0.11352868 -1.02610699 -1.06038321  0.53827224 -0.66581843
   -1.23511514 -0.21825161 -1.62925834]]

 [[-0.11352868 -0.9225242  -1.06038321  0.4051263  -0.66581843
   -1.23511514 -0.21825161 -1.57681737]
  [-0.11352868 -1.02610699 -1.06038321  0.53827224 -0.66581843
   -1.23511514 -0.21825161 -1.62925834]
  [-0.11352868 -1.12968978 -1.06038321  0.71580015 -0.66581843
   -1.23511514 -0.21825161 -1.65922462]]

 [[-0.11352868 -1.02610699 -1.06038321  0.53827224 -0.66581843
   -1.23511514 -0.21825161 -1.62925834]
  [-0.11352868 -1.12968978 -1.06038321  0.71580015 -0.66581843
   -1.23511514 -0.21825161 -1.65922462]
  [-0.11352868 -0.9225242  -1.06038321  0.80456411 -0.66581843
   -1.23511514 -0.21825161 -1.68919089]]], shape=(3, 3, 8), dtype=float64)

tf.Tensor([0. 0. 0.], 

In [8]:
pd.concat([SNE_X_train, SNE_y_train], axis=1).head(10)

,0,1,2,3,4,5,6,7,8
0,-0.113529,-0.093862,-0.682515,0.094452,-0.665818,-1.235115,-0.218252,-1.539360,0.0
1,-0.113529,-0.922524,-1.060383,0.405126,-0.665818,-1.235115,-0.218252,-1.576817,0.0
2,-0.113529,-1.026107,-1.060383,0.538272,-0.665818,-1.235115,-0.218252,-1.629258,0.0
3,-0.113529,-1.129690,-1.060383,0.715800,-0.665818,-1.235115,-0.218252,-1.659225,0.0
4,-0.113529,-0.922524,-1.060383,0.804564,-0.665818,-1.235115,-0.218252,-1.689191,0.0
5,-0.113529,-1.026107,-1.060383,0.937710,-0.665818,-1.235115,-0.218252,-1.719157,0.0
6,-0.113529,-0.922524,-1.060383,0.893328,-0.665818,-1.235115,-0.218252,-1.734140,0.0
7,-0.113529,-0.922524,-1.060383,0.982092,0.549517,-1.235115,-0.333181,-1.741632,6000.0
8,-0.113529,-1.129690,-1.060383,0.538272,1.764852,0.769233,-0.226741,-1.636750,16000.0
9,-0.113529,-0.922524,-1.060383,0.005688,1.764852,-0.984572,-0.322080,-1.329596,24000.0


In [9]:
# 위 내용을 함수 형태로 만들고 싶은 경우

def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(x).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(y[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [10]:
train_data = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SNE_X_test, SNE_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [11]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 8), (28,)


In [12]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dropout(0.3),
  Dense(1)
])

In [13]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dropout(0.3),
  Dense(1)
])

In [14]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dropout(0.3),
  Dense(1)
])

In [15]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [16]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('./Control_Weights/dropout_group/new_data/new_multi_stne_lstm_weight_dropout.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('./Control_Weights/dropout_group/new_data/new_multi_stne_lstm_weight_ns_dropout.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('./Control_Weights/dropout_group/new_data/new_multi_stne_rnn_weight_dropout.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('./Control_Weights/dropout_group/new_data/new_multi_stne_rnn_weight_ns_dropout.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('./Control_Weights/dropout_group/new_data/new_multi_stne_gru_weight_dropout.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('./Control_Weights/dropout_group/new_data/new_multi_stne_gru_weight_ns_dropout.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [17]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500
    496/Unknown - 9s 6ms/step - loss: 7622.1572 - mse: 183492544.0000 - mae: 7622.5474
Epoch 1: val_loss improved from inf to 7099.40820, saving model to ./Control_Weights/dropout_group/new_data\new_multi_stne_lstm_weight_dropout.h5
498/498 [==============================] - 13s 12ms/step - loss: 7617.5415 - mse: 183437456.0000 - mae: 7617.9312 - val_loss: 7099.4082 - val_mse: 170949632.0000 - val_mae: 7099.8281 - lr: 0.0010
Epoch 2/500
488/498 [============================>.] - ETA: 0s - loss: 7554.3857 - mse: 180923904.0000 - mae: 7554.7832
Epoch 2: val_loss improved from 7099.40820 to 7015.66650, saving model to ./Control_Weights/dropout_group/new_data\new_multi_stne_lstm_weight_dropout.h5
498/498 [==============================] - 3s 7ms/step - loss: 7551.6519 - mse: 180609952.0000 - mae: 7552.0493 - val_loss: 7015.6665 - val_mse: 167314288.0000 - val_mae: 7016.0498 - lr: 0.0010
Epoch 3/500
492/498 [============================>.] - ETA: 0s - loss: 7437.3813 - mse: 1759

In [18]:
lstm_save_path = f"./Control_Models/dropout_group/new_data/lstm_model"

save_model(lstm_model, lstm_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/dropout_group/new_data/lstm_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/dropout_group/new_data/lstm_model\assets


In [19]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500
    487/Unknown - 5s 4ms/step - loss: 7617.5449 - mse: 183643664.0000 - mae: 7617.9116
Epoch 1: val_loss improved from inf to 7093.78369, saving model to ./Control_Weights/dropout_group/new_data\new_multi_stne_rnn_weight_dropout.h5
498/498 [==============================] - 7s 6ms/step - loss: 7614.1650 - mse: 183299472.0000 - mae: 7614.5327 - val_loss: 7093.7837 - val_mse: 170595184.0000 - val_mae: 7094.1895 - lr: 0.0010
Epoch 2/500
484/498 [============================>.] - ETA: 0s - loss: 7549.7144 - mse: 180706448.0000 - mae: 7550.1069
Epoch 2: val_loss improved from 7093.78369 to 6993.23682, saving model to ./Control_Weights/dropout_group/new_data\new_multi_stne_rnn_weight_dropout.h5
498/498 [==============================] - 2s 4ms/step - loss: 7534.3335 - mse: 179901872.0000 - mae: 7534.7266 - val_loss: 6993.2368 - val_mse: 166404000.0000 - val_mae: 6993.6611 - lr: 0.0010
Epoch 3/500
490/498 [============================>.] - ETA: 0s - loss: 7381.7129 - mse: 17390374

In [20]:
rnn_save_path = f"./Control_Models/dropout_group/new_data/rnn_model"

save_model(rnn_model, rnn_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/dropout_group/new_data/rnn_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/dropout_group/new_data/rnn_model\assets


In [21]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500
    496/Unknown - 12s 7ms/step - loss: 7617.5737 - mse: 183572128.0000 - mae: 7617.9424
Epoch 1: val_loss improved from inf to 7111.22852, saving model to ./Control_Weights/dropout_group/new_data\new_multi_stne_gru_weight_dropout.h5
498/498 [==============================] - 14s 10ms/step - loss: 7621.4521 - mse: 183647680.0000 - mae: 7621.8208 - val_loss: 7111.2285 - val_mse: 171451440.0000 - val_mae: 7111.6177 - lr: 0.0010
Epoch 2/500
498/498 [==============================] - ETA: 0s - loss: 7582.9624 - mse: 182010224.0000 - mae: 7583.3530
Epoch 2: val_loss improved from 7111.22852 to 7060.93408, saving model to ./Control_Weights/dropout_group/new_data\new_multi_stne_gru_weight_dropout.h5
498/498 [==============================] - 3s 7ms/step - loss: 7582.9624 - mse: 182010224.0000 - mae: 7583.3530 - val_loss: 7060.9341 - val_mse: 169298384.0000 - val_mae: 7061.3418 - lr: 0.0010
Epoch 3/500
495/498 [============================>.] - ETA: 0s - loss: 7531.9551 - mse: 17957

In [22]:
gru_save_path = f"./Control_Models/dropout_group/new_data/gru_model"

save_model(gru_model, gru_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/dropout_group/new_data/gru_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/dropout_group/new_data/gru_model\assets


In [23]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 2056.536865234375
lstm_history Validation MSE: 14990351.0
lstm_history Validation MAE: 2056.980224609375
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 2131.15625
lstm_history_ns Validation MSE: 16024292.0
lstm_history_ns Validation MAE: 2131.528564453125
-------------rnn_history-------------
rnn_history Validation Loss: 2097.125
rnn_history Validation MSE: 15523258.0
rnn_history Validation MAE: 2097.495849609375
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 2196.435791015625
rnn_history_ns Validation MSE: 16901166.0
rnn_history_ns Validation MAE: 2196.804931640625
-------------gru_history-------------
gru_history Validation Loss: 2267.94091796875
gru_history Validation MSE: 16917126.0
gru_history Validation MAE: 2268.437744140625
-------------gru_history_ns-------------
gru_history_ns Validation Loss: 2370.127685546875
gru_history_ns Validation MSE: 18066224.0
gr

In [24]:
St_NotEncode_Basic_data = pd.read_pickle("Basic_StandardScalar_final_data")
  
print(St_NotEncode_Basic_data.shape)

(52578, 9)


In [25]:
# Feature와 Label 분리하기
def Feature_Label(datafile) :
    X = datafile.iloc[:,:-1]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SB_X, SB_y = Feature_Label(St_NotEncode_Basic_data)
print(SNE_X.shape, SNE_y.shape)
SB_X_train, SB_X_test, SB_y_train, SB_y_test = train_test_split(SB_X, SB_y, test_size=0.2, random_state=10, shuffle=False)
print(SB_X_train.shape, SB_X_test.shape, SB_y_train.shape, SB_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 8) (17422,)
(42062, 8) (10516, 8) (42062,) (10516,)


In [26]:
def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(x).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(y[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [27]:
train_data = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SB_X_test, SB_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [28]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 8), (28,)


In [29]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dropout(0.3),
  Dense(1)
])

In [30]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dropout(0.3),
  Dense(1)
])

In [31]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dropout(0.3),
  Dense(1)
])

In [32]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [33]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('./Control_Weights/dropout_group/base_data/base_multi_stne_lstm_weight_dropout.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('./Control_Weights/dropout_group/base_data/base_multi_stne_lstm_weight_ns_dropout.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('./Control_Weights/dropout_group/base_data/base_multi_stne_rnn_weight_dropout.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('./Control_Weights/dropout_group/base_data/base_multi_stne_rnn_weight_ns_dropout.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('./Control_Weights/dropout_group/base_data/base_multi_stne_gru_weight_dropout.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('./Control_Weights/dropout_group/base_data/base_multi_stne_gru_weight_ns_dropout.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [34]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500
   1500/Unknown - 21s 8ms/step - loss: 2.9313 - mse: 37.9603 - mae: 3.3104
Epoch 1: val_loss improved from inf to 3.07867, saving model to ./Control_Weights/dropout_group/base_data\base_multi_stne_lstm_weight_dropout.h5
1503/1503 [==============================] - 25s 11ms/step - loss: 2.9324 - mse: 37.9764 - mae: 3.3115 - val_loss: 3.0787 - val_mse: 37.2547 - val_mae: 3.4837 - lr: 0.0010
Epoch 2/500
1503/1503 [==============================] - ETA: 0s - loss: 2.2499 - mse: 23.1272 - mae: 2.6261
Epoch 2: val_loss improved from 3.07867 to 2.73749, saving model to ./Control_Weights/dropout_group/base_data\base_multi_stne_lstm_weight_dropout.h5
1503/1503 [==============================] - 12s 8ms/step - loss: 2.2499 - mse: 23.1272 - mae: 2.6261 - val_loss: 2.7375 - val_mse: 30.9625 - val_mae: 3.1456 - lr: 0.0010
Epoch 3/500
1502/1503 [============================>.] - ETA: 0s - loss: 2.1937 - mse: 22.2046 - mae: 2.5738
Epoch 3: val_loss did not improve from 2.73749
1503/1503 [

In [35]:
lstm_save_path = f"./Control_Models/dropout_group/base_data/lstm_model"

save_model(lstm_model, lstm_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/dropout_group/base_data/lstm_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/dropout_group/base_data/lstm_model\assets


In [36]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500
   1499/Unknown - 9s 3ms/step - loss: 3.4486 - mse: 54.2113 - mae: 3.8263
Epoch 1: val_loss improved from inf to 3.36046, saving model to ./Control_Weights/dropout_group/base_data\base_multi_stne_rnn_weight_dropout.h5
1503/1503 [==============================] - 10s 4ms/step - loss: 3.4498 - mse: 54.2386 - mae: 3.8275 - val_loss: 3.3605 - val_mse: 44.5988 - val_mae: 3.7681 - lr: 0.0010
Epoch 2/500
1501/1503 [============================>.] - ETA: 0s - loss: 2.7845 - mse: 35.8460 - mae: 3.1670
Epoch 2: val_loss did not improve from 3.36046
1503/1503 [==============================] - 6s 4ms/step - loss: 2.7840 - mse: 35.8388 - mae: 3.1666 - val_loss: 3.3966 - val_mse: 46.6880 - val_mae: 3.8200 - lr: 0.0010
Epoch 3/500
1496/1503 [============================>.] - ETA: 0s - loss: 2.6938 - mse: 33.1235 - mae: 3.0840
Epoch 3: val_loss did not improve from 3.36046
1503/1503 [==============================] - 6s 4ms/step - loss: 2.6993 - mse: 33.2385 - mae: 3.0896 - val_loss: 3.38

In [37]:
rnn_save_path = f"./Control_Models/dropout_group/base_data/rnn_model"

save_model(rnn_model, rnn_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/dropout_group/base_data/rnn_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/dropout_group/base_data/rnn_model\assets


In [38]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500
   1495/Unknown - 17s 7ms/step - loss: 3.1496 - mse: 43.8582 - mae: 3.5268
Epoch 1: val_loss improved from inf to 3.31152, saving model to ./Control_Weights/dropout_group/base_data\base_multi_stne_gru_weight_dropout.h5
1503/1503 [==============================] - 20s 8ms/step - loss: 3.1499 - mse: 43.8501 - mae: 3.5272 - val_loss: 3.3115 - val_mse: 41.0331 - val_mae: 3.7196 - lr: 0.0010
Epoch 2/500
1496/1503 [============================>.] - ETA: 0s - loss: 2.4586 - mse: 27.7568 - mae: 2.8387
Epoch 2: val_loss improved from 3.31152 to 3.02492, saving model to ./Control_Weights/dropout_group/base_data\base_multi_stne_gru_weight_dropout.h5
1503/1503 [==============================] - 11s 7ms/step - loss: 2.4592 - mse: 27.7757 - mae: 2.8393 - val_loss: 3.0249 - val_mse: 36.2732 - val_mae: 3.4271 - lr: 0.0010
Epoch 3/500
1502/1503 [============================>.] - ETA: 0s - loss: 2.4431 - mse: 27.7889 - mae: 2.8280
Epoch 3: val_loss improved from 3.02492 to 2.55994, saving mo

In [39]:
gru_save_path = f"./Control_Models/dropout_group/base_data/gru_model"

save_model(gru_model, gru_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/dropout_group/base_data/gru_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/dropout_group/base_data/gru_model\assets


In [40]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 2.1455907821655273
lstm_history Validation MSE: 21.38953971862793
lstm_history Validation MAE: 2.5433478355407715
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 2.0507092475891113
lstm_history_ns Validation MSE: 19.947450637817383
lstm_history_ns Validation MAE: 2.453434705734253
-------------rnn_history-------------
rnn_history Validation Loss: 2.5279994010925293
rnn_history Validation MSE: 28.53551483154297
rnn_history Validation MAE: 2.9417717456817627
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 2.7464778423309326
rnn_history_ns Validation MSE: 31.365253448486328
rnn_history_ns Validation MAE: 3.152616500854492
-------------gru_history-------------
gru_history Validation Loss: 2.1761653423309326
gru_history Validation MSE: 21.44436264038086
gru_history Validation MAE: 2.582608938217163
-------------gru_history_ns-------------
gru_history_ns Validation Loss: 2